## KNN的原理及使用

分类算法

通过你的“邻居”来推断你的类别

### 1. 定义：

如果一个样本在特征空间中的**k个最相似**(即特征空间中最邻近)的样本中的**大多数**属于某一个类别，则该样本也属于这个类别。

### 2. 如何计算距离

- 欧式距离

  $A(a_1,...a_i)$ $B(b_1,...,b_i)$
  
  $d = \sqrt{(a_1-b_i)^2 + ... + (a_i-b_i)^2}$

### 3. API调用

**`sklearn.neighbors.KNeighborsClassifier(n_neighbors=5, algorithm='auto')`**

 n_neighbors：int,可选（默认= 5），k_neighbors查询默认使用的邻居数 

algorithm：{‘auto’，‘ball_tree’，‘kd_tree’，‘brute’}，可选用于计算最近邻居的算法：‘ball_tree’将会使BallTree，‘kd_tree’将使用 KDTree。‘auto’将尝试根据传递给fit方法的值来决定最合适的算法。 (不同实现方式影响效率)!

加快搜索速度——基于算法的改进KDTree,API接口里面有实现

> 需要对特征进行进行标准化

### 4. 案例：鸢尾花分类

In [21]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [28]:
ir = load_iris() 

# 获取特征值
x = ir.data
# 获取目标值
y = ir.target
# 划分数据集
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.25)
# 特征工程（标准化）
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

In [29]:
# 算法流程
knn = KNeighborsClassifier()

# # 模型训练
# knn = KNeighborsClassifier(n_neighbors = 5)
# knn.fit(x_train,y_train)

# # 对测试集数据进行分类
# y_predict = knn.predict(x_test)
# y_predict

In [9]:
# # 计算准确率
# knn.score(x_test, y_test)

0.9736842105263158

In [10]:
# print("各类别的精确率与召回率：")
# print(classification_report(y_test, y_predict, target_names=ir.target_names))

各类别的精确率与召回率：
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        12
  versicolor       0.92      1.00      0.96        11
   virginica       1.00      0.93      0.97        15

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38



In [19]:
import sklearn
# 查看能够按什么指标进行模型选择
print(sorted(sklearn.metrics.SCORERS.keys()))

['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'completeness_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'max_error', 'mutual_info_score', 'neg_brier_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_gamma_deviance', 'neg_mean_poisson_deviance', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'neg_root_mean_squared_error', 'normalized_mutual_info_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'rand_score', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc', 'roc_auc_ovo', 'roc_auc_ovo_weighted', 'roc_auc_ovr', 'roc_auc_ovr_weighted', 'top_k_accuracy', 'v_measure_score']


In [30]:
# 网格搜索

# 构建参数
param = {"n_neighbors": [3, 5, 10]}

gc = GridSearchCV(knn, param_grid=param, cv=5, scoring='accuracy')
gc.fit(x_train, y_train)

# 最高的accuary
print(gc.best_score_)
# 最高的accuary对应的参数
print(gc.best_params_)

0.9648221343873518
{'n_neighbors': 10}


In [33]:
# 按n为10进行建模

# 模型训练
knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(x_train,y_train)

# 对测试集数据进行分类
y_predict = knn.predict(x_test)

# 计算准确率
print("准确率：", knn.score(x_test, y_test))
print(" ")

print("各类别的精确率与召回率：")
print(classification_report(y_test, y_predict, target_names=ir.target_names))

准确率： 0.9736842105263158
 
各类别的精确率与召回率：
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        16
  versicolor       1.00      0.92      0.96        12
   virginica       0.91      1.00      0.95        10

    accuracy                           0.97        38
   macro avg       0.97      0.97      0.97        38
weighted avg       0.98      0.97      0.97        38



### 5. 模型问题

- k值的选定：

    - k值取很小：容易受异常点影响

    - k值取很大：容易受k值数量（类别）波动

- 性能问题

### 6. 优缺点

优点：
- 简单，易于理解，易于实现，无需估计参数，无需训练

缺点：
- 懒惰算法，对测试样本分类时的计算量大，内存开销大
- 必须指定K值，K值选择不当则分类精度不能保证

### 7. 使用场景

使用场景：小数据场景，几千～几万样本，具体场景具体业务去测试

### 8. 网格搜索

**超参数：** 通常情况下，有很多参数是需要手动指定的（如k-近邻算法中的K值），这种叫超参数。

但是手动过程繁杂，所以需要对模型预设几种超参数组合。每组超参数都采用**交叉验证**来进行评估。最后选出最优参数组合建立模型。


**`sklearn.model_selection.GridSearchCV`** 


***sklearn.model_selection.GridSearchCV(estimator,param_grid=None,cv=None,scoring=)*** 对估计器的指定参数值进行详尽搜索

- 参数解释：
    - estimator：估计器对象
    - param_grid：估计器参数(dict){“n_neighbors”:[1,3,5]}
    - cv：指定几折交叉验证
    - scoring：按什么指标进行模型选择
- 使用方法：
    - fit：输入训练数据
    - score：计算准确率
- 属性
    - best_score_:在交叉验证中测试的最好结果
    - best_estimator_：最好的参数模型
    - cv_results_:每次交叉验证后的测试集准确率结果和训练集准确率结果